In [1]:
!git clone https://github.com/RF5/simple-speaker-embedding.git

Cloning into 'simple-speaker-embedding'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 119 (delta 65), reused 70 (delta 28), pack-reused 0
Receiving objects: 100% (119/119), 1.22 MiB | 10.07 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [2]:
import torch
# for the new model
model = torch.hub.load('RF5/simple-speaker-embedding', 'convgru_embedder')
# for the original model
#model = torch.hub.load('RF5/simple-speaker-embedding', 'gru_embedder')
model.eval()

/opt/conda/lib/python3.10/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/RF5/simple-speaker-embedding/zipball/master" to /root/.cache/torch/hub/master.zip
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_ma

ConvGRUEmbedder(
  (model): ConvRNNEmbedder(
    (conv_encoder): ConvEncoder(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GroupNorm(512, 512, eps=1e-05, affine=True)
          (3): GELU(approximate='none')
        )
        (1-4): 4 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
        (5-6): 2 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
      )
    )
    (rnn): GRU(512, 768, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
    (head): Linear(in_features=1536, out_features=256, bias=True)
  )
)

In [3]:
#mel = model.melspec_from_file('example.wav')
# if you were using the `convgru_embedder`, you can go:
import librosa

In [4]:
wav, _ = librosa.load('/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/spk_11-11_22_1_0_d1_ch5.wav', sr=16000)
wav = torch.from_numpy(wav).float()
embedding1 = model(wav[None])

In [5]:
wav, _ = librosa.load('/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/spk_12-12_10_0_0_d6_ch5.wav', sr=16000)
wav = torch.from_numpy(wav).float()
embedding2 = model(wav[None])

In [6]:
test_embedding_normalized = embedding1[0] / torch.linalg.norm(embedding1.cpu())
average_embedding_normalized = embedding2[0] / torch.linalg.norm(embedding2.cpu())

similarity = torch.dot(test_embedding_normalized, average_embedding_normalized) / ((torch.dot(test_embedding_normalized, test_embedding_normalized) * torch.dot(average_embedding_normalized, average_embedding_normalized)) ** 0.5)
similarity = (similarity + 1) / 2
print(similarity)

tensor(0.4910, device='cuda:0', grad_fn=<DivBackward0>)


In [7]:
import os
import numpy as np

emb_lst = np.zeros((75,256))
speaker_to_file_map = dict()

for dirname, _, filenames in os.walk('/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment'):
    for filename in filenames:
        name = filename.split('-')
        spk_id = name[0]
        array = speaker_to_file_map.get(spk_id, -1)
        sample = os.path.join(dirname, filename)

        if array==-1:
            speaker_to_file_map[spk_id] = [str(sample)]
        else:
            speaker_to_file_map[spk_id].append(str(sample))

In [8]:
import pandas as pd
df = pd.read_csv("/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/signle-channel-trials.trl", sep="\t", header=None)

In [10]:
        speaker, test_file = df.iloc[89887][0], df.iloc[89887][1]
        
        wav, _ = librosa.load(f"/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/test/{test_file}.wav", sr=16000)
        wav = torch.from_numpy(wav).float()
        test_embedding = model(wav[None])

        enrollment_list = speaker_to_file_map.get(speaker) 
        
        wav, _ = librosa.load(enrollment_list[0], sr=16000)
        wav = torch.from_numpy(wav).float()
        e_embedding_1 = model(wav[None])

        wav, _ = librosa.load(enrollment_list[1], sr=16000)
        wav = torch.from_numpy(wav).float()
        e_embedding_2 = model(wav[None])

        wav, _ = librosa.load(enrollment_list[2], sr=16000)
        wav = torch.from_numpy(wav).float()
        e_embedding_3 = model(wav[None])
        
        average_embedding = torch.mean(torch.stack([e_embedding_1, e_embedding_2, e_embedding_3], dim=0), dim=0).squeeze()

        test_embedding_normalized = test_embedding / torch.linalg.norm(test_embedding.cpu())
        average_embedding_normalized = average_embedding / torch.linalg.norm(average_embedding.cpu())

        similarity = torch.dot(test_embedding_normalized[0], average_embedding_normalized) / ((torch.dot(test_embedding_normalized[0], test_embedding_normalized[0]) * torch.dot(average_embedding_normalized, average_embedding_normalized)) ** 0.5)
        similarity = (similarity + 1) / 2
        
        print(speaker,test_file,similarity)

spk_9 99744cf108 tensor(0.5599, device='cuda:0', grad_fn=<DivBackward0>)


In [11]:
with open('/kaggle/working/output.txt', 'w') as output:
    for i in range(89888,len(df)):
        speaker, test_file = df.iloc[i][0], df.iloc[i][1]
        
        wav, _ = librosa.load(f"/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/test/{test_file}.wav", sr=16000)
        wav = torch.from_numpy(wav).float()
        test_embedding = model(wav[None])

        enrollment_list = speaker_to_file_map.get(speaker) 
        
        wav, _ = librosa.load(enrollment_list[0], sr=16000)
        wav = torch.from_numpy(wav).float()
        e_embedding_1 = model(wav[None])

        wav, _ = librosa.load(enrollment_list[1], sr=16000)
        wav = torch.from_numpy(wav).float()
        e_embedding_2 = model(wav[None])

        wav, _ = librosa.load(enrollment_list[2], sr=16000)
        wav = torch.from_numpy(wav).float()
        e_embedding_3 = model(wav[None])
        
        average_embedding = torch.mean(torch.stack([e_embedding_1, e_embedding_2, e_embedding_3], dim=0), dim=0).squeeze()

        test_embedding_normalized = test_embedding / torch.linalg.norm(test_embedding.cpu())
        average_embedding_normalized = average_embedding / torch.linalg.norm(average_embedding.cpu())

        similarity = torch.dot(test_embedding_normalized[0], average_embedding_normalized) / ((torch.dot(test_embedding_normalized[0], test_embedding_normalized[0]) * torch.dot(average_embedding_normalized, average_embedding_normalized)) ** 0.5)
        similarity = (similarity + 1) / 2
        
        if similarity>0.7:
            print(speaker,test_file,similarity)

        # similarity = cosine_similarity(test_embedding.cpu(), average_embedding.cpu())[0,0]

        output.write(f"{speaker}\t{test_file}\t{similarity}\n")

spk_9 696ff15708 tensor(0.7167, device='cuda:0', grad_fn=<DivBackward0>)
spk_9 3c513cc46a tensor(0.7143, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 f7d7e75a32 tensor(0.7198, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 5cf5c2c126 tensor(0.7071, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 f6464e2ffc tensor(0.7023, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 eca606a9ee tensor(0.7110, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 aadcde18f1 tensor(0.7071, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 6aeb3e8468 tensor(0.7002, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 ed7ad751ea tensor(0.7717, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 5c06a0fe22 tensor(0.7233, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 a4fc529e0b tensor(0.7213, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 8c48c6239e tensor(0.7431, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 76ae2301c1 tensor(0.7230, device='cuda:0', grad_fn=<DivBackward0>)
spk_10 9b7b4fc815 tensor(0.7410, device=

In [ ]:
'''import os
import numpy as np

emb_lst = np.zeros((75,256))
spk_lst = []

for dirname, _, filenames in os.walk('/kaggle/input/robovox-sp-cup-2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment'):
    for filename in filenames:
        name = filename.split('-')
        spk_id = name[0]
        num = spk_id.split('_')
        spk_num = int(num[1])-1
        
        sample = os.path.join(dirname, filename)
        #mel = model.melspec_from_file(sample)
        wav, _ = librosa.load(sample, sr=16000)
        wav = torch.from_numpy(wav).float()
        embedding = model(wav[None])
            
        print(filename)'''